# P2. Les vues SQL

## Définition d'une vue SQL  

Une vue n'est rien de plus qu'une instruction SQLite qui est stockée dans la base de données avec un nom associé. Il s'agit en fait d'une composition d'une table sous la forme d'une requête SQLite prédéfinie.

Une vue peut contenir toutes les lignes d'une table ou des lignes sélectionnées d'une ou plusieurs tables. Une vue peut être créée à partir d'une ou plusieurs tables qui dépendent de la requête SQLite écrite pour créer une vue.

## Code utilitaire

In [1]:
import os
import sqlite3
import pandas as pd   

In [2]:
from pathlib import Path

In [3]:
# Ouvre connection vers SQLite db
chemin_bdd = Path('db') / 'chinook.db'
conn = sqlite3.connect(chemin_bdd)

In [4]:
# Fonction permettant d'exécuter un requête SQL sur une BDD définie par sa connexion conn
def executer_requete(requete_sql, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(requete_sql)
        conn.commit()
    except sqlite3.Error as e:
        print("Erreur lors de l'execution de la requête")
        print(e)
        return
    cursor.close()


----------

## Example de vue SQL: v_albums

La requête suivante permet d'obtenir une liste (simplifiée) des albums / artistes correspondants:  

In [5]:
# Requete sur les tables albums & artists
requete = """
    SELECT AlbumId,
           Title,
           a.Name AS Artist
    FROM albums
    JOIN artists a on albums.ArtistId = a.ArtistId
    ORDER BY Artist;
"""
# utilisation de la fonction de pandas "read_sql_quert"
# elle permet d'afficher et de formater graphiquement les résultats
albums = pd.read_sql_query(requete, conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


In [6]:
len(albums)

347

Le code ci-dessous permet de créer une vue correspondant à la requête ci-dessous :    
Référence vue sous SQLite : https://www.sqlite.org/lang_createview.html

In [7]:
# creation vue albums avec artistes
creation_vue_albums = """
    CREATE VIEW v_albums (
        AlbumId,
        Title,
        Artist)
    AS
        SELECT AlbumId,
               Title,
               a.Name AS Artist
        FROM albums
        JOIN artists a on albums.ArtistId = a.ArtistId
        ORDER BY Artist, Title;
"""

executer_requete(creation_vue_albums, conn)

Une fois la vue créé, on peut la réutiliser dans des requêtes SQL comme une table:  

In [8]:
# Utilisation de la vue v_albums
albums = pd.read_sql_query("SELECT * FROM v_albums LIMIT 10;", conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


In [9]:
len(albums)

10

In [10]:
# Utilisation de la vue v_albums avec une clause WHERE
albums = pd.read_sql_query("SELECT * FROM v_albums WHERE Artist = 'U2';", conn)
albums

,AlbumId,Title,Artist
0,232,Achtung Baby,U2
1,233,All That You Can't Leave Behind,U2
2,234,B-Sides 1980-1990,U2
3,235,How To Dismantle An Atomic Bomb,U2
4,255,Instant Karma: The Amnesty International Campa...,U2
5,236,Pop,U2
6,237,Rattle And Hum,U2
7,238,The Best Of 1980-1990,U2
8,239,War,U2
9,240,Zooropa,U2


In [11]:
len(albums)

10

In [12]:
# Utilisation de la vue v_albums avec une autre jointure
requete = """
    SELECT a.AlbumId,
           a.Title,
           a.Artist,
           count(t.TrackId) as Tracks
    FROM v_albums a
    JOIN tracks t ON a.AlbumId = t.AlbumId
    GROUP BY 1, 2, 3
    ORDER BY 3, 2;
"""
albums = pd.read_sql_query(requete, conn)
albums.head()

,AlbumId,Title,Artist,Tracks
0,1,For Those About To Rock We Salute You,AC/DC,10
1,4,Let There Be Rock,AC/DC,8
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1
3,267,Worlds,Aaron Goldberg,1
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,2


In [13]:
len(albums)

347

In [14]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS v_albums;", conn)

### Question: Avantages & Inconvénients des vues SQL

Quelles sont les avantages et inconvénients des vues SQL ?  
*A compléter*

---

## Exercices de création / mise en oeuvre de vues

### Vue 1: caractéristiques complètes des Albums  
A partir de l'exemple de vue ci-dessus, créer et exploiter une nouvelle vue v_albums reprenant pour chaque album:  
- AlbumId
- Title
- Artist
- nb de pistes
- Durée totale de l'album (en minutes)
- Prix de l'album

In [15]:
# création de la vue
creation_vue_albums = """
CREATE VIEW info_albums AS 
SELECT a.AlbumId , a.Title AS 'Titre Album',  
		a2.Name,
		COUNT(t.TrackId) AS 'Nombre de piste', SUM(t.Milliseconds)/1000/60 AS 'Durée',
		SUM(t.UnitPrice) AS 'Prix' 
FROM albums a
LEFT JOIN artists a2 ON a.ArtistId = a2.ArtistId 
JOIN tracks t ON a.AlbumId = t.AlbumId 
GROUP BY a.AlbumId""" 

# appel de la fonction avec ma vue et ma variable conn en paramètre
executer_requete(creation_vue_albums, conn)

In [16]:
albums = pd.read_sql_query("SELECT * FROM info_albums LIMIT 10;", conn)
albums.head(5)

,AlbumId,Titre Album,Name,Nombre de piste,Durée,Prix
0,1,For Those About To Rock We Salute You,AC/DC,10,40,9.90
1,2,Balls to the Wall,Accept,1,5,0.99
2,3,Restless and Wild,Accept,3,14,2.97
3,4,Let There Be Rock,AC/DC,8,40,7.92
4,5,Big Ones,Aerosmith,15,73,14.85


In [17]:
len(albums)

10

In [18]:
# ex d'utilsation de la vue
# la vue permet de créer une nouvelle table
requete = """SELECT *
from info_albums 
where Name = 'AC/DC'"""

albums = pd.read_sql_query(requete, conn)
albums.head(5)

,AlbumId,Titre Album,Name,Nombre de piste,Durée,Prix
0,1,For Those About To Rock We Salute You,AC/DC,10,40,9.90
1,4,Let There Be Rock,AC/DC,8,40,7.92


In [19]:
len(albums)

2

In [20]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS info_albums;", conn)

### Vue 2: caractéristiques détaillées des pistes
Créer et exploiter une nouvelle vue v_tracks reprenant pour chaque piste:  
- TrackId
- Name
- Genre
- Album
- Artist
- Composer
- Temps (en seconde)
- MediaType

In [21]:
creation_vue_pistes = """CREATE VIEW info_pistes AS 
SELECT t.TrackId , t.Name AS 'Piste', g.Name AS 'Genre', a2.Title AS 'ALBUM', t.Composer AS 'Compositeur',
		a3.Name AS 'Artiste', t.Milliseconds/1000 AS 'Durée en secondes', mt.Name AS 'Type'
FROM  tracks t
INNER JOIN genres g ON t.GenreId = g.GenreId
INNER JOIN media_types mt ON t.MediaTypeId = mt.MediaTypeId 
INNER JOIN albums a2 ON t.AlbumId = a2.AlbumId 
INNER JOIN artists a3 ON a2.ArtistId = a3.ArtistId """

executer_requete(creation_vue_pistes, conn)

In [22]:
pistes = pd.read_sql_query("""select *
from info_pistes""", conn)
pistes.head(5)

,TrackId,Piste,Genre,ALBUM,Compositeur,Artiste,Durée en secondes,Type
0,1,For Those About To Rock (We Salute You),Rock,For Those About To Rock We Salute You,"Angus Young, Malcolm Young, Brian Johnson",AC/DC,343,MPEG audio file
1,6,Put The Finger On You,Rock,For Those About To Rock We Salute You,"Angus Young, Malcolm Young, Brian Johnson",AC/DC,205,MPEG audio file
2,7,Let's Get It Up,Rock,For Those About To Rock We Salute You,"Angus Young, Malcolm Young, Brian Johnson",AC/DC,233,MPEG audio file
3,8,Inject The Venom,Rock,For Those About To Rock We Salute You,"Angus Young, Malcolm Young, Brian Johnson",AC/DC,210,MPEG audio file
4,9,Snowballed,Rock,For Those About To Rock We Salute You,"Angus Young, Malcolm Young, Brian Johnson",AC/DC,203,MPEG audio file


In [23]:
len(pistes)

3503

In [24]:
requete = """SELECT *
from info_pistes 
where Artiste = 'AC/DC'"""

pistes = pd.read_sql_query(requete, conn)
pistes.head(5)

,TrackId,Piste,Genre,ALBUM,Compositeur,Artiste,Durée en secondes,Type
0,1,For Those About To Rock (We Salute You),Rock,For Those About To Rock We Salute You,"Angus Young, Malcolm Young, Brian Johnson",AC/DC,343,MPEG audio file
1,6,Put The Finger On You,Rock,For Those About To Rock We Salute You,"Angus Young, Malcolm Young, Brian Johnson",AC/DC,205,MPEG audio file
2,7,Let's Get It Up,Rock,For Those About To Rock We Salute You,"Angus Young, Malcolm Young, Brian Johnson",AC/DC,233,MPEG audio file
3,8,Inject The Venom,Rock,For Those About To Rock We Salute You,"Angus Young, Malcolm Young, Brian Johnson",AC/DC,210,MPEG audio file
4,9,Snowballed,Rock,For Those About To Rock We Salute You,"Angus Young, Malcolm Young, Brian Johnson",AC/DC,203,MPEG audio file


In [25]:
len(pistes)

18

In [26]:
executer_requete("DROP VIEW IF EXISTS info_pistes;", conn)

### Vue 3: Vue augmentée invoices
A. Créer une nouvelle vue v_invoices à partir de la table invoices ajoutant 3 nouveaux attributs basés sur l'attribut `InvoiceDate`:  
- semaine de la transaction
- mois de la transaction
- année de la transaction

In [27]:
creation_vue_dates = """CREATE VIEW info_dates AS
SELECT i.InvoiceId, SUBSTR(i.InvoiceDate , 0, 5) AS 'Année', SUBSTR(i.InvoiceDate, 6, 2) AS 'Month', strftime('%W', i.InvoiceDate) AS 'Week'
FROM invoices i 
group by i.InvoiceId"""

executer_requete(creation_vue_dates, conn)

In [28]:
requete = """SELECT InvoiceID, Année, Month, Week
FROM info_dates 
WHERE Week BETWEEN  '09' AND  '25'"""

i_date = pd.read_sql_query(requete, conn)
i_date.head(5)

,InvoiceId,Année,Month,Week
0,14,2009,03,09
1,15,2009,03,09
2,16,2009,03,09
3,17,2009,03,09
4,18,2009,03,10


In [29]:
len(i_date)

130

B. Exploiter la vue v_invoices ainsi crée pour déterminer les ventes annuelles par pays.

In [30]:
requete = """SELECT DISTINCT(id.Année), i.BillingCountry, ii.UnitPrice * COUNT(i.BillingCountry) AS 'Income' 
FROM invoices i
LEFT JOIN info_dates id ON i.InvoiceId = id.InvoiceId 
LEFT JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId 
GROUP BY i.BillingCountry, Année
ORDER BY Année DESC """

i_billing = pd.read_sql_query(requete, conn)
i_billing.head(10)

,Année,BillingCountry,Income
0,2013,Argentina,24.75
1,2013,Austria,0.99
2,2013,Belgium,5.94
3,2013,Brazil,37.62
4,2013,Canada,72.27
5,2013,Czech Republic,24.75
6,2013,Denmark,8.91
7,2013,Finland,15.84
8,2013,France,40.59
9,2013,Germany,9.90


In [31]:
len(i_billing)

101

In [32]:
executer_requete("DROP VIEW IF EXISTS info_dates;", conn)

### Vue 4: A vous de jouer
Créer et exploiter une nouvelle vue de votre choix  

In [33]:
# création d'une vue qui recense le pays des employés
creation_vue_country = """
CREATE VIEW info_Country AS
SELECT e.City ,e.State ,e.Country 
FROM employees e ;
"""
# lancement de la fonction qui me permet d'exécuter la requete
executer_requete(creation_vue_country, conn)

Erreur lors de l'execution de la requête
table info_Country already exists


In [34]:
# requete pour définir quelles sont les pays des clients où les employés
# n'y vivent pas (exportation)
requete = """SELECT c.Country
FROM customers c
left join info_Country ic
WHERE c.Country NOT IN 
(SELECT ic.Country
FROM info_Country )
GROUP BY c.Country ;"""

i_country = pd.read_sql_query(requete, conn)
i_country.head(20)

,Country
0,Argentina
1,Australia
2,Austria
3,Belgium
4,Brazil
5,Chile
6,Czech Republic
7,Denmark
8,Finland
9,France


In [35]:
len(i_country)

23

In [36]:
executer_requete("DROP VIEW IF EXISTS info_Country;", conn)